In [1]:
# Pitch Recognition Forked From https://github.com/justinsalamon/audio_to_midi_melodia
import os
import math
import numpy as np
import pretty_midi as midi
from pylab import *
import librosa             # The librosa library
import librosa.display     # librosa's display module (for plotting features)
import IPython.display     # IPython's display module (for in-line audio)
import matplotlib.pyplot as plt # matplotlib plotting functions
import matplotlib.style as ms   # plotting style
import pitch_rec

In [2]:
infile = "test2.mp3"
outfile = "melody.mid"
bpm = 120
high_mapper = np.array([67, 69, 71, 72, 74, 76])
male_mapper = np.array([60, 68, 67, 64, 62, 60])
female_mapper = np.array([69, 62, 63, 60, 67, 69])

In [3]:
pitch_rec.audio_to_midi_melodia(infile, outfile, bpm, hop=128, smooth=0.1)

Loading audio...
Extracting melody f0 with MELODIA...
Converting Hz to MIDI notes...
Saving MIDI to disk...
Conversion complete.


In [4]:
data = midi.PrettyMIDI("./melody.mid")
pitches = list(map(lambda x: x.pitch, data.instruments[0].notes))
high_pitch = high_mapper[high_mapper <= max(pitches)]
high_pitch = high_mapper[0] if len(high_pitch) == 0 else high_pitch[-1]
idx = np.where(high_mapper == high_pitch)[0][0]
pitch = male_mapper[idx]
pitch = 67

In [5]:
def generate_wave(pitch, sr=44100, am=-2.5, length=2.):
    base = 440 * 2 **((pitch-69)/12)
    F1 = base * 2**(-1/12)
    F2 = F1 * 2**(-3/12)
    F3 = F2 * 2**(7/12)
    F4 = F3 * 2**(-5/12)
    c1 = [sin(2 * pi * F1 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)] 
    c2 = [sin(2 * pi * F2 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)]
    c3 = [sin(2 * pi * F3 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)]
    c4 = [sin(2 * pi * F4 * n) * exp(am * n) for n in np.arange(0., length, 1. / sr)]
    return c1+c2+c3+c4+c1+c2+c3+c4
sr = 44100
prog = np.array(generate_wave(pitch, sr))
IPython.display.Audio(data=prog, rate=sr) # press the "play" button to hear audio
librosa.output.write_wav('test.wav', prog, sr)